<a href="https://colab.research.google.com/github/pmb06d/SM_Cinemax_Matrix/blob/master/SM_Matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Authenticate the Google SDK and bring in the data

In [0]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#!pip install --upgrade -q gspread

import pandas as pd

# This package can read google sheets
import gspread


gc = gspread.authorize(GoogleCredentials.get_application_default())
sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1LgVqtPS-xlU_w3EPHlzjc53B78QS321owqJe2Y1qZSQ/edit?ts=5d4357f8#gid=0')

# july 'https://docs.google.com/spreadsheets/d/1LgVqtPS-xlU_w3EPHlzjc53B78QS321owqJe2Y1qZSQ/edit?ts=5d4357f8#gid=0'

# aug 'https://docs.google.com/spreadsheets/d/1d1vH8RLG6sycVh-2E1EJeYWGvc9VsIEcn8LUUsbtpsQ/edit?ts=5d4357ae#gid=619122347'

In [0]:
import re

def find_matrix(test_list):
  temp = []
  for i in test_list:
    try:
      temp.append(re.findall("\'(.+Matrix)\'",str(i))[0])
    except:
      continue
  return(temp[0])

find_matrix(sh.worksheets())

'July SM Matrix'

In [0]:
sh.worksheet(find_matrix(sh.worksheets()))

<Worksheet 'July SM Matrix' id:0>

In [0]:
import numpy as np

# Put into a dataframe
matrix = pd.DataFrame(sh.worksheet(find_matrix(sh.worksheets())).get_all_values())
del matrix.index.name
matrix = matrix.replace('NA',np.nan)

matrix = matrix.replace('', np.nan).dropna(how='all').dropna(axis=1, thresh =2)
#matrix = matrix.iloc[0:len(matrix)-2,:]

matrix.head()

,1,2,3,4,5,6,7,8,9,10,14,15,16,17
1,COLOR TAG GUIDE,FECHA DE PUBLICACIÓN,PLATAFORMA,NaN,OBJETIVO,BUDGET,NaN,BOOST DATE,TÍTULO,TIPO DE CONTENIDO,COPY,NaN,NOTES,BITLY (LAT)
3,NaN,NaN,FACEBOOK,INSTAGRAM,NaN,FACEBOOK,INSTAGRAM,NaN,NaN,NaN,FACEBOOK,INSTAGRAM,NaN,URL
4,NaN,1 Jul,Facebook,Instagram,Engagement,$200,NaN,SAME DAY,LEGO BATMAN,COPA AMÉRICA (gif),"Quando inesperadamente, esta temporada de fute...","Quando inesperadamente, esta temporada de fute...",NaN,NaN
5,NaN,NaN,NaN,NaN,Video Views,$300,NaN,SAME DAY,MATRIX,Teaser,"Há vinte anos, descobrimos que toda nossa real...","Há vinte anos, descobrimos que toda nossa real...",NaN,NaN
6,NaN,NaN,Facebook,Instagram,Engagement,$200,NaN,SAME DAY,LEGO BATMAN,COPA AMÉRICA (gif),"Cuando inesperadamente, esta temporada de fútb...","Cuando inesperadamente, esta temporada de fútb...",NaN,bit.ly/LegoBatmanenCinemaxLA


In [0]:
def replicate_down(raw_list):

  column_names = []

  for i in range(len(raw_list)):
    if str(raw_list[i]) == 'nan':
      if str(raw_list[i-1]) == 'nan' and i != 0:
        column_names.append(column_names[-1])
      else:
        column_names.append(raw_list[i-1])
    else:
      column_names.append(raw_list[i])

  return(column_names)

In [0]:
index_df = pd.DataFrame(list(zip(list(matrix.iloc[0,:]),list(matrix.iloc[1,:])))).dropna(how='all')
index_df[0] = replicate_down(list(index_df[0]))
index = list(index_df.itertuples(index=False, name=None))

new_index = []
for tup in index:
  if str(tup[1]) == 'nan':
    new_index.append(tup[0])
  else:
    new_index.append(tup[0]+' - '+tup[1])

index_df['Label'] = new_index
new_index

['COLOR TAG GUIDE',
 'FECHA DE PUBLICACIÓN ',
 'PLATAFORMA - FACEBOOK',
 'PLATAFORMA - INSTAGRAM',
 'OBJETIVO',
 'BUDGET - FACEBOOK',
 'BUDGET - INSTAGRAM',
 'BOOST DATE',
 'TÍTULO ',
 'TIPO DE CONTENIDO',
 'COPY - FACEBOOK',
 'COPY - INSTAGRAM',
 'NOTES',
 'BITLY (LAT) - URL']

In [0]:
index_df

,0,1,Label
0,COLOR TAG GUIDE,NaN,COLOR TAG GUIDE
1,FECHA DE PUBLICACIÓN,NaN,FECHA DE PUBLICACIÓN
2,PLATAFORMA,FACEBOOK,PLATAFORMA - FACEBOOK
3,PLATAFORMA,INSTAGRAM,PLATAFORMA - INSTAGRAM
4,OBJETIVO,NaN,OBJETIVO
5,BUDGET,FACEBOOK,BUDGET - FACEBOOK
6,BUDGET,INSTAGRAM,BUDGET - INSTAGRAM
7,BOOST DATE,NaN,BOOST DATE
8,TÍTULO,NaN,TÍTULO
9,TIPO DE CONTENIDO,NaN,TIPO DE CONTENIDO


In [0]:
label_dict = dict(zip(index_df['Label'],index_df[0]))
label_dict

{'BITLY (LAT) - URL': 'BITLY (LAT)',
 'BOOST DATE': 'BOOST DATE',
 'BUDGET - FACEBOOK': 'BUDGET',
 'BUDGET - INSTAGRAM': 'BUDGET',
 'COLOR TAG GUIDE': 'COLOR TAG GUIDE',
 'COPY - FACEBOOK': 'COPY',
 'COPY - INSTAGRAM': 'COPY',
 'FECHA DE PUBLICACIÓN ': 'FECHA DE PUBLICACIÓN ',
 'NOTES': 'NOTES',
 'OBJETIVO': 'OBJETIVO',
 'PLATAFORMA - FACEBOOK': 'PLATAFORMA',
 'PLATAFORMA - INSTAGRAM': 'PLATAFORMA',
 'TIPO DE CONTENIDO': 'TIPO DE CONTENIDO',
 'TÍTULO ': 'TÍTULO '}

In [0]:
platforms = list(set(replicate_down(list(matrix.iloc[1,:]))))

def clean_list(list_of_stuff, list_to_remove):
  for i in list_to_remove:
    try:
      list_of_stuff.remove(i)
    except:
      continue
  return(list_of_stuff)

platforms = clean_list(platforms, ['VISITS','URL',np.nan])

platforms

['FACEBOOK', 'INSTAGRAM']

In [0]:
df_list = []

for i in platforms:
  df_list.append(list())

p_labels = []
for i in platforms:
  p_labels.append(list(index_df.loc[index_df[1]==i,'Label']))

g_labels = list(index_df.loc[index_df[1].isna(),'Label'])
g_labels = g_labels[1:]

b_labels = list(index_df.loc[index_df[0]=='BUDGET','Label'])

#platform: p_labels
#general: g_labels
#budget columns: b_labels

g_labels

['FECHA DE PUBLICACIÓN ',
 'OBJETIVO',
 'BOOST DATE',
 'TÍTULO ',
 'TIPO DE CONTENIDO',
 'NOTES']

In [0]:
p_labels

[['PLATAFORMA - FACEBOOK', 'BUDGET - FACEBOOK', 'COPY - FACEBOOK'],
 ['PLATAFORMA - INSTAGRAM', 'BUDGET - INSTAGRAM', 'COPY - INSTAGRAM']]

In [0]:
b_labels

['BUDGET - FACEBOOK', 'BUDGET - INSTAGRAM']

In [0]:
matrix2 = matrix.iloc[2:len(matrix),0:len(new_index)]
matrix2.columns = new_index
matrix2 = matrix2.iloc[:,1:len(list(matrix2))]

matrix2.reset_index(drop=True, inplace=True)

matrix2.tail()

,FECHA DE PUBLICACIÓN,PLATAFORMA - FACEBOOK,PLATAFORMA - INSTAGRAM,OBJETIVO,BUDGET - FACEBOOK,BUDGET - INSTAGRAM,BOOST DATE,TÍTULO,TIPO DE CONTENIDO,COPY - FACEBOOK,COPY - INSTAGRAM,NOTES,BITLY (LAT) - URL
96,NaN,Facebook,Instagram,Engagement,NaN,NaN,SAME DAY,N/A,INSTRUCCIONES PARA,NaN,NaN,NaN,NaN
97,30 Jul,Facebook,Instagram,Engagement,$200,NaN,SAME DAY,SULLY,DETRÁS DE : THOM HANKS,Você sabia que existe um asteróide com o nome ...,NaN,NaN,NaN
98,NaN,Facebook,Instagram,Engagement,$200,NaN,SAME DAY,SULLY,DETRÁS DE : THOM HANKS,¿Sabías que existe un asteroide con el nombre ...,NaN,NaN,NaN
99,NaN,NaN,NaN,Total Budget LAT:,#REF!,#REF!,NaN,OVERSPEND,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,Total Budget BR:,"$7,650",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
def find_string(column_name):
  for i in list(matrix2.loc[matrix2[column_name].notna(),column_name].unique()):
    if i.startswith('$'):
      continue
    else:
      return(i)

remove = (matrix2.loc[matrix2[b_labels[0]]==find_string(b_labels[0]),b_labels[0]].index)
last_row = int(matrix2.loc[remove,:].reset_index().iloc[:,0])

matrix2 = matrix2.iloc[0:last_row,:]

matrix2.tail()

,FECHA DE PUBLICACIÓN,PLATAFORMA - FACEBOOK,PLATAFORMA - INSTAGRAM,OBJETIVO,BUDGET - FACEBOOK,BUDGET - INSTAGRAM,BOOST DATE,TÍTULO,TIPO DE CONTENIDO,COPY - FACEBOOK,COPY - INSTAGRAM,NOTES,BITLY (LAT) - URL
19,NaN,Facebook,Instagram,Video Views,$300,NaN,SAME DAY,SAFE,Teaser,Quem ousou trair Jason Statham vai sentir as c...,NaN,NaN,NaN
20,NaN,Facebook,Instagram,Video Views,$300,$50,SAME DAY,HORRIBLE BOSSES,Promo,Los grandes amigos son capaces hasta de matar ...,Los grandes amigos son capaces hasta de matar ...,NaN,bit.ly/QuieroMataraMiJefeenCinemaxLA
21,NaN,Facebook,Instagram,Video Views,$300,$50,SAME DAY,SULLY,SULLY (premier),"La historia la cuenta cualquiera, pero la escr...","La historia la cuenta cualquiera, pero la escr...",NaN,bit.ly/SullyenCinemaxLA
22,NaN,Facebook,Instagram,Video Views,$350,$50,SAME DAY,THE SPACE BETWEEN US,PROMO,Descubre un amor de otro planeta. 🛰\n\nEl Espa...,Descubre un amor de otro planeta. 🛰\n\nEl Espa...,NaN,bit.ly/ElEspacioEntreNosotrosenCinemaxLA
23,7 Jul,Facebook,Instagram,Brand Awareness,$250,NaN,SAME DAY,MATRIX,COPA AMÉRICA (video),Em busca de algo muito mais divertido que fute...,Em busca de algo muito mais divertido que fute...,NaN,NaN


In [0]:
print('NAs by Column:','\n')
for i in list(matrix2):
  print(i+':'+str(len(matrix2.loc[matrix2[i].isna(),i])))

NAs by Column: 

FECHA DE PUBLICACIÓN :17
PLATAFORMA - FACEBOOK:1
PLATAFORMA - INSTAGRAM:9
OBJETIVO:0
BUDGET - FACEBOOK:0
BUDGET - INSTAGRAM:21
BOOST DATE:0
TÍTULO :0
TIPO DE CONTENIDO:0
COPY - FACEBOOK:0
COPY - INSTAGRAM:8
NOTES:23
BITLY (LAT) - URL:11


In [0]:
# populate all rows on the date column
matrix3 = matrix2.copy()
matrix3.iloc[:,0] = replicate_down(list(matrix2.iloc[:,0]))

matrix3.tail()

,FECHA DE PUBLICACIÓN,PLATAFORMA - FACEBOOK,PLATAFORMA - INSTAGRAM,OBJETIVO,BUDGET - FACEBOOK,BUDGET - INSTAGRAM,BOOST DATE,TÍTULO,TIPO DE CONTENIDO,COPY - FACEBOOK,COPY - INSTAGRAM,NOTES,BITLY (LAT) - URL
19,6 Jul,Facebook,Instagram,Video Views,$300,NaN,SAME DAY,SAFE,Teaser,Quem ousou trair Jason Statham vai sentir as c...,NaN,NaN,NaN
20,6 Jul,Facebook,Instagram,Video Views,$300,$50,SAME DAY,HORRIBLE BOSSES,Promo,Los grandes amigos son capaces hasta de matar ...,Los grandes amigos son capaces hasta de matar ...,NaN,bit.ly/QuieroMataraMiJefeenCinemaxLA
21,6 Jul,Facebook,Instagram,Video Views,$300,$50,SAME DAY,SULLY,SULLY (premier),"La historia la cuenta cualquiera, pero la escr...","La historia la cuenta cualquiera, pero la escr...",NaN,bit.ly/SullyenCinemaxLA
22,6 Jul,Facebook,Instagram,Video Views,$350,$50,SAME DAY,THE SPACE BETWEEN US,PROMO,Descubre un amor de otro planeta. 🛰\n\nEl Espa...,Descubre un amor de otro planeta. 🛰\n\nEl Espa...,NaN,bit.ly/ElEspacioEntreNosotrosenCinemaxLA
23,7 Jul,Facebook,Instagram,Brand Awareness,$250,NaN,SAME DAY,MATRIX,COPA AMÉRICA (video),Em busca de algo muito mais divertido que fute...,Em busca de algo muito mais divertido que fute...,NaN,NaN


In [0]:
df_split = []
for i in range(len(platforms)):
  df = matrix3.loc[:,g_labels+p_labels[i]]
  df.rename(columns=label_dict, inplace=True)
  df_split.append(df)

In [0]:
df_split[0].head()

,FECHA DE PUBLICACIÓN,OBJETIVO,BOOST DATE,TÍTULO,TIPO DE CONTENIDO,NOTES,PLATAFORMA,BUDGET,COPY
0,1 Jul,Engagement,SAME DAY,LEGO BATMAN,COPA AMÉRICA (gif),NaN,Facebook,$200,"Quando inesperadamente, esta temporada de fute..."
1,1 Jul,Video Views,SAME DAY,MATRIX,Teaser,NaN,NaN,$300,"Há vinte anos, descobrimos que toda nossa real..."
2,1 Jul,Engagement,SAME DAY,LEGO BATMAN,COPA AMÉRICA (gif),NaN,Facebook,$200,"Cuando inesperadamente, esta temporada de fútb..."
3,1 Jul,Video Views,SAME DAY,MATRIX,Teaser,NaN,Facebook,$300,"Hace 20 años, descubrimos que nuestra realidad..."
4,1 Jul,Video Views,SAME DAY,THE TOWN,Teaser,NaN,Facebook,$300,"En este trabajo, el amor puede ser un error mo..."


In [0]:
final_df = pd.concat(df_split)
final_df = final_df.loc[final_df['BUDGET'].notna(),:]
final_df['PLATAFORMA'] = replicate_down(list(final_df['PLATAFORMA']))

final_df = final_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

final_df.head()

,FECHA DE PUBLICACIÓN,OBJETIVO,BOOST DATE,TÍTULO,TIPO DE CONTENIDO,NOTES,PLATAFORMA,BUDGET,COPY
0,1 Jul,Engagement,SAME DAY,LEGO BATMAN,COPA AMÉRICA (gif),NaN,Facebook,$200,"Quando inesperadamente, esta temporada de fute..."
1,1 Jul,Video Views,SAME DAY,MATRIX,Teaser,NaN,Facebook,$300,"Há vinte anos, descobrimos que toda nossa real..."
2,1 Jul,Engagement,SAME DAY,LEGO BATMAN,COPA AMÉRICA (gif),NaN,Facebook,$200,"Cuando inesperadamente, esta temporada de fútb..."
3,1 Jul,Video Views,SAME DAY,MATRIX,Teaser,NaN,Facebook,$300,"Hace 20 años, descubrimos que nuestra realidad..."
4,1 Jul,Video Views,SAME DAY,THE TOWN,Teaser,NaN,Facebook,$300,"En este trabajo, el amor puede ser un error mo..."


In [0]:
from google.colab import files

# e.g. save pandas output as csv
final_df.to_csv('SM Matrix.csv', index=False)

files.download('SM Matrix.csv')